In [1]:
import pandas as pd
import numpy as np
def ipdx(a,b,k):
    step = (b-a)/float(k+1)
    return [a] + [a + (n*step) for n in range(1,k+1)]

In [2]:
dataIn = pd.read_csv('LST_Nakhon_Krabi.csv', header=0, skiprows=0)
dataIn = dataIn.sort_values(['Year','Day'])
dataIn['addrcode'] = dataIn['addrcode'].astype(str)
dataIn['Day'] = dataIn['Day'].astype(str)
dataIn['Day'] = dataIn['Day'].apply(lambda x: x.zfill(3))
dataIn["YearDay"] = dataIn["Year"].map(str) + dataIn["Day"]
dataIn.drop('Unnamed: 0', axis =1, inplace= True)

In [3]:
LSTYear = 2014
LSTweekly = pd.DataFrame()
df3 = pd.DataFrame()

In [78]:
NSTData = dataIn.loc[dataIn['addrcode'].str[:2] == '81']
for i in range(3): 
    LSTYear = LSTYear + i
    NSTData_Y1 = pd.DataFrame(NSTData.loc[dataIn['Year'] == (LSTYear)])
    print(LSTYear)

    NSTData_Y1.drop('Day', axis =1, inplace= True)
    NSTData_Y1.drop('Year', axis =1, inplace= True)

    NSTData_Y1= NSTData_Y1.pivot(index='addrcode', columns='YearDay', values='temperature').reset_index().rename_axis(None, axis=1)

    m = NSTData_Y1.mean(axis=1)
    for i, col in enumerate(NSTData_Y1):
        NSTData_Y1.iloc[:, i] = NSTData_Y1.iloc[:, i].fillna(m)

    NSTData_Y1 = pd.DataFrame(NSTData_Y1)
    addrcode = NSTData_Y1.iloc[:,0]
    addrcode = addrcode.astype(str)

    NSTData_Y1.drop('addrcode',axis =1, inplace= True)

    movingaveragedf = NSTData_Y1.rolling(window=2,axis=1).mean()
    movingaveragedf.iloc[:,0] = NSTData_Y1.iloc[:,0]
    movingaveragedf = pd.DataFrame(movingaveragedf.interpolate())
    movingaveragedf = movingaveragedf.round(2)


    out = []
    for i in range(len(movingaveragedf.index)):
        row = []
        for j in range(len(movingaveragedf.columns[0:45])): # 001-361 should be 45
            begin = movingaveragedf.iloc[i,j]
            end = movingaveragedf.iloc[i,j+1]
            interval = 7
        
    #       Linear Interpolation
            row = row + ipdx(a = begin, b = end, k = interval)
    #    Manully Compute 201x361 - 201yx365
            if j==44:
                begin = movingaveragedf.iloc[i,44]
                end = movingaveragedf.iloc[i,45]
            
                # Linear Interpolation
                row = row + ipdx(a = begin, b = end, k =3) 
        
        row = row + [end]
        out.append(row)
    results = pd.DataFrame(out)    


    for i in range(len(results.columns)+1):
        colheadYear = str(LSTYear)
        colheaddaynum = str(i).rjust(3,'0')
        results.columns.values[i-1] = colheadYear + colheaddaynum
        results._data = results._data.rename_axis(lambda x: x,0, True)

    colHeads = list(results)

    colHeads = colHeads[1:]
    colName = ['colHeads']
    LSTdf = pd.DataFrame(colHeads, columns = colName)

    LSTdf['colHeads'] = LSTdf['colHeads'].apply(str)
    LSTdf['Year'] = LSTdf.colHeads.str[0:4]

    LSTdf['DayNum'] = LSTdf.colHeads.str[4:7]
    LSTdf['LSTDate'] = (pd.to_datetime(LSTdf['Year']))
    

    LSTdf['offset'] = LSTdf['DayNum'].astype(int)-1

    LSTdf['current'] = LSTdf.apply(lambda x: x['LSTDate'] + pd.DateOffset(days=x['offset']),axis=1)
    LSTdf['Weekno'] = LSTdf['current'].dt.week
    dayonerow = LSTdf.offset == 0
    LSTdf.loc[dayonerow,'Weekno']= 1
    LSTdf['Weekno'] = LSTdf['Weekno'].astype(int)
    LSTdf['Year'] = LSTdf['Year'].astype(int)
    colNames = ('Year', 'Weekno')
    cols = [col for col in LSTdf.columns if col in colNames]
    

    # 2016 Quick fix
    if (LSTYear == 2016):
        LSTdf.iloc[[0,1],6] = 52
     

    tempcolHeads = pd.DataFrame(colHeads)
    tempcolHeads.columns = {'colHeads'}
    LSTdf['colHeads'] = pd.Series(tempcolHeads['colHeads'])
    colHeadYearWeek = LSTdf.groupby(['Year','Weekno'])['colHeads'].apply(list)
    resultdf = pd.DataFrame({'Year':LSTdf.loc[:,'Year'], 'Weekno':LSTdf.loc[:,'Weekno']})
    weeknum = resultdf.drop_duplicates()
    weeknum = weeknum.reset_index(drop=True)
    weekLST = pd.DataFrame(np.zeros((len(colHeadYearWeek), len(results))))

    for (idx,row) in results.iterrows(): 
        rowidx = -1
        for x in colHeadYearWeek:
            rowidx = rowidx +1
            weekLST.iloc[rowidx,idx]= row[x].mean()
    weekLST.columns = [i+1 for i in range(weekLST.shape[1])]
    weekLST.columns = [ str(col) for col in addrcode]
    weekdata = pd.concat([weeknum, weekLST], axis=1)
    
    df3 = pd.concat([df3, weekdata], axis=0, ignore_index=True)
    #df3 = LSTweekly.append(weekdata, ignore_index=True)
    LSTweekly = df3
    #LSTweekly
    LSTweekly['Weekno'] = LSTweekly[['Weekno','Year']].astype(str).sum(axis=1)
    LSTweekly['Weekno'] = LSTweekly['Weekno'].astype(str).str[:-2]
    LSTweekly = LSTweekly.drop('Year',axis = 1)
    LSTweekly.set_index('Weekno').T
    LSTweekly.to_csv('LSTweekly_Krabi_'+str(LSTYear)+'.csv', encoding = 'utf-8')

2017
